In [ ]:
pip install pandas


: 

In [ ]:
import verification_library as veri   
from peft import PeftConfig, PeftModel
from transformers import LlamaForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os  
import importlib
import gc
from dotenv import load_dotenv
from collections import defaultdict
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

importlib.reload(veri)

counter_greater_than_200 = 0

contract_lines_4X = []
contract_lines_8X = []

contracts_4X = veri.get_files("/home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated/0.4.x/contracts/dataset")
contracts_8X = veri.get_files("/home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated/0.8.x/contracts/dataset/arithmetic") 

for contract_path, contract_name in contracts_4X:
    contract, line = veri.find_occurrences(contract_path, "// <yes> <report>")
    if contract.count('\n') > 200:
        counter_greater_than_200 += 1 
    else:
        clean_contract = veri.replace_lines_with_string(contract, line, '')
        contract_lines_4X.append((contract_name, clean_contract, line))

for contract_path, contract_name in contracts_8X:
    contract, line = veri.find_occurrences(contract_path, "// <yes> <report>")
    
    if contract.count('\n') > 200:
        counter_greater_than_200 += 1
    else:
        clean_contract = veri.replace_lines_with_string(contract, line, '')
        contract_lines_8X.append((contract_name, clean_contract,line))

print(counter_greater_than_200, len(contracts_4X), len(contracts_8X))
veri.print_json_report("reports/contract_no_comment8X_2.json", contract_lines_8X)
veri.print_txt_report("reports/contract_no_comment8x_2.txt", contract_lines_8X)

veri.print_json_report("reports/contract_no_comment4X_2.json", contract_lines_4X)
veri.print_txt_report("reports/contract_no_comment4x_2.txt", contract_lines_4X)


: 

In [ ]:
importlib.reload(veri)
contracts_with_results = veri.read_json_report("reports/contract_with_results_20_8X.json") #list of (replaced_contract, line, require)
contract_lines = veri.read_json_report("reports/contract_no_comment8X_2.json")  ##list of (contract_name, contract, line)

In [ ]:
importlib.reload(veri)

##VL

contract = contract_lines[1]

for contract_patch in contracts_with_results[contract[0]]:    
        no_test, result = veri.evaluate_contracts(contract, contract_patch)
        print(result)


In [ ]:
importlib.reload(veri)

##Precondition
contract = contract_lines[1]
contract_code = contract[1]
lines = contract[2]
precondition = lines["pre"] +1 
require = contracts_with_results[contract[0]][0][2]

contract_patch = veri.insert_empty_line(contract_code, precondition)
contract_patch = veri.replace_lines_with_string(contract_patch, [precondition], require) 

patch = (contract_patch, precondition, require)
no_test, result = veri.evaluate_contracts(contract, patch)
print(result)

In [ ]:
importlib.reload(veri)

##Postcondition
contract = contract_lines[1]
contract_code = contract[1]
lines = contract[2]
precondition, postcondition = veri.find_function_bounds(contract[1], contract_patch_VL[1])
precondition = precondition + 1

require = contracts_with_results[contract[0]][0][2]

contract_patch = veri.insert_empty_line(contract_code, postcondition)
contract_patch = veri.replace_lines_with_string(contract_patch, [postcondition], require)
print(contract_patch)
patch = (contract_patch, postcondition, require)
no_test, result = veri.evaluate_contracts(contract, patch)
print(result)

In [ ]:
#Pre+Post
contract = contract_lines[1]
contract_code = contract[1]
lines = contract[2]
postcondition = lines["post"] 
precondition = lines["pre"] +1

require = contracts_with_results[contract[0]][0][2]

contract_patch = veri.insert_empty_line(contract_code, postcondition)

contract_patch = veri.replace_lines_with_string(contract_patch, [postcondition], require)

 
contract_patch = veri.insert_empty_line(contract_patch, precondition)

contract_patch = veri.replace_lines_with_string(contract_patch, [precondition], require)
print(contract_patch)

patch = (contract_patch, postcondition, require)
no_test, result = veri.evaluate_contracts(contract, patch)
print(result)

In [ ]:
##Pre+Post+VL

contract = contract_lines[4]
contract_code = contract[1]
lines = contract[2]
postcondition = lines["post"] 
precondition = lines["pre"] +1

require = contracts_with_results[contract[0]][0][2]

for contract_patched_VL in contracts_with_results[contract[0]]:  
    contract_patch = contract_patched_VL[0]
    if postcondition != contract_patch[1]: 
        contract_patch = veri.insert_empty_line(contract_patch, postcondition)
        contract_patch = veri.replace_lines_with_string(contract_patch, [postcondition], require)
    if precondition != contract_patch[1]: 
        contract_patch = veri.insert_empty_line(contract_patch, precondition)
        contract_patch= veri.replace_lines_with_string(contract_patch, [precondition], require) 
   
    print(contract_patch)
    patch_lines = f"{precondition},{contract_patched_VL[1]},{postcondition}"
    patch = (contract_patch, patch_lines, require)
    no_test, result = veri.evaluate_contracts(contract, patch)
    print(result)




In [ ]:
##Pre+VL
contract = contract_lines[4]
contract_code = contract[1]
lines = contract[2]
precondition = lines["pre"] +1

require = contracts_with_results[contract[0]][0][2]

for contract_patched_VL in contracts_with_results[contract[0]]:  
    contract_patch = contract_patched_VL[0]
    if precondition != contract_patch[1]: 
        contract_patch = veri.insert_empty_line(contract_patch, precondition)
        contract_patch= veri.replace_lines_with_string(contract_patch, [precondition], require) 

    patch_lines = f"{precondition},{contract_patched_VL[1]}"
    patch = (contract_patch, patch_lines, require)
    no_test, result = veri.evaluate_contracts(contract, patch)
    print(result)

In [ ]:
#Post+VL
contract = contract_lines[4]
contract_code = contract[1]
lines = contract[2]
postcondition = lines["post"] 


require = contracts_with_results[contract[0]][0][2]

for contract_patched_VL in contracts_with_results[contract[0]]:  
    contract_patch = contract_patched_VL[0]
    if postcondition != contract_patch[1]: 
        contract_patch = veri.insert_empty_line(contract_patch, postcondition)
        contract_patch = veri.replace_lines_with_string(contract_patch, [postcondition], require)
    print(contract_patch)
    patch_lines = f"{contract_patched_VL[1]},{postcondition}"
    patch = (contract_patch, patch_lines, require)
    no_test, result = veri.evaluate_contracts(contract, patch)
    print(result)

In [ ]:
contracts_with_results = veri.read_json_report("reports/contract_with_results_llama_8X.json") #list of (replaced_contract, line, require)
contract_lines = veri.read_json_report("reports/contract_no_comment8X.json")  ##list of (contract_name, contract, line)

In [ ]:
importlib.reload(veri)
headers = [
    'Solidity_file_name',
    'vulnerability_type',
    'vulnerable_line_number',
    'vulnerability_function_entry_line',
    'vulnerability_function_end_line',
    'VL',
    'pre_post_',
    'pre',
    'post',
    'pre_VL_post',
    'pre_VL',
    'VL_post'
]

veri.create_csv_if_not_exists('/home/matteo/FLAMES/verification-results/sb-heists/reports/validation_results_llama_8X.csv', headers)

for contract in contract_lines: 
        contract_name = contract[0]
        contract_code = contract[1]
        lines = contract[2]
        require = contracts_with_results[contract[0]][0][2]
        
        for contract_patch_VL in contracts_with_results[contract[0]]:  
                contract_patch = contract_patch_VL[0]
                VL = contract_patch_VL[1]
                precondition, postcondition = veri.find_function_bounds(contract[1], contract_patch_VL[1])
                precondition = precondition + 1
                # VL  
                no_test, result_VL = veri.evaluate_contracts(contract, contract_patch_VL)
                if no_test:
                        break

                # Pre
                contract_patch_pre = veri.insert_empty_line(contract_code, precondition)
                contract_patch_pre = veri.replace_lines_with_string(contract_patch_pre, [precondition], require) 
                patch_pre = (contract_patch_pre, precondition, require)
                no_test, result_pre = veri.evaluate_contracts(contract, patch_pre)

                # Post
                contract_patch_post = veri.insert_empty_line(contract_code, postcondition)
                contract_patch_post = veri.replace_lines_with_string(contract_patch_post, [postcondition], require)
                patch_post = (contract_patch_post, postcondition, require)
                no_test, result_post = veri.evaluate_contracts(contract, patch_post)
                

                # Pre + Post
                contract_patch_pre_post = veri.insert_empty_line(contract_code, postcondition)
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [postcondition], require)
                contract_patch_pre_post = veri.insert_empty_line(contract_patch_pre_post, precondition)
                
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [precondition ], require) 
                
                patch_lines = f"{precondition},{postcondition}"
                patch_pre_post = (contract_patch_pre_post, patch_lines, require)
                no_test, result_pre_post = veri.evaluate_contracts(contract, patch_pre_post)
             
                # Pre + VL + Post (PVP)
                if postcondition != VL: 
                        contract_patch_PVP = veri.insert_empty_line(contract_patch, postcondition)
                        contract_patch_PVP = veri.replace_lines_with_string(contract_patch_PVP, [postcondition], require)
                if precondition != VL: 
                        contract_patch_PVP = veri.insert_empty_line(contract_patch_PVP, precondition)
                        contract_patch_PVP= veri.replace_lines_with_string(contract_patch_PVP, [precondition], require) 

                patch_lines = f"{precondition},{contract_patch_VL[1]},{postcondition}"
                patch_PVP = (contract_patch_PVP, patch_lines, require)
                no_test, result_PVP = veri.evaluate_contracts(contract, patch_PVP)

                # Pre + VL (PV)
                if precondition != VL: 
                        contract_patch_PV = veri.insert_empty_line(contract_patch, precondition)
                        contract_patch_PV= veri.replace_lines_with_string(contract_patch_PV, [precondition], require) 

                patch_lines = f"{precondition},{contract_patch_VL[1]}"
                patch_PV = (contract_patch_PV, patch_lines, require)
                no_test, result_PV = veri.evaluate_contracts(contract, patch_PV)

                # VL + Post (VP)
                if postcondition != VL: 
                        contract_patch_VP = veri.insert_empty_line(contract_patch, postcondition)
                        contract_patch_VP = veri.replace_lines_with_string(contract_patch_VP, [postcondition], require)

                patch_lines = f"{contract_patch_VL[1]},{postcondition}"
                patch_VP = (contract_patch_VP, patch_lines, require)
                no_test, result_VP = veri.evaluate_contracts(contract, patch_VP)

            
                type_V = veri.get_directory_name(contract_name,"/home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated")
              

                row = {
                        'Solidity_file_name': contract_name,
                        'vulnerability_type': type_V,
                        'vulnerable_line_number': contract_patch_VL[1],
                        'vulnerability_function_entry_line': precondition - 1,
                        'vulnerability_function_end_line': postcondition,
                        'VL': result_VL,
                        'pre_post_': result_pre_post,
                        'pre': result_pre,
                        'post': result_post,
                        'pre_VL_post': result_PVP,
                        'pre_VL': result_PV,
                        'VL_post': result_VP
                }

                veri.append_row('/home/matteo/FLAMES/verification-results/sb-heists/reports/validation_results_llama_8X.csv', headers, row)
###plausible_patch

        

In [21]:
contracts_with_results = veri.read_json_report("reports/contract_with_results_20_8X.json") #list of (replaced_contract, line, require)
contract_lines = veri.read_json_report("reports/contract_no_comment8X_2.json")  ##list of (contract_name, contract, line)

In [22]:
importlib.reload(veri)
headers = [
    'Solidity_file_name',
    'vulnerability_type',
    'vulnerable_line_number',
    'vulnerability_function_entry_line',
    'vulnerability_function_end_line',
    'VL',
    'pre_post_',
    'pre',
    'post',
    'pre_VL_post',
    'pre_VL',
    'VL_post'
]

veri.create_csv_if_not_exists('/home/matteo/FLAMES/verification-results/sb-heists/reports/validation_results_20_8X.csv', headers)

for contract in contract_lines: 
        contract_name = contract[0]
        contract_code = contract[1]
        lines = contract[2]
        require = contracts_with_results[contract[0]][0][2]
        
        for contract_patch_VL in contracts_with_results[contract[0]]:  
                contract_patch = contract_patch_VL[0]
                precondition, postcondition = veri.find_function_bounds(contract[1], contract_patch_VL[1])
                precondition = precondition + 1
                # VL  
                no_test, result_VL = veri.evaluate_contracts(contract, contract_patch_VL)
                if no_test:
                        break

                # Pre
                contract_patch_pre = veri.insert_empty_line(contract_code, precondition)
                contract_patch_pre = veri.replace_lines_with_string(contract_patch_pre, [precondition], require) 
                patch_pre = (contract_patch_pre, precondition, require)
                no_test, result_pre = veri.evaluate_contracts(contract, patch_pre)

                # Post
                contract_patch_post = veri.insert_empty_line(contract_code, postcondition)
                contract_patch_post = veri.replace_lines_with_string(contract_patch_post, [postcondition], require)
                patch_post = (contract_patch_post, postcondition, require)
                no_test, result_post = veri.evaluate_contracts(contract, patch_post)

                # Pre + Post
                contract_patch_pre_post = veri.insert_empty_line(contract_code, postcondition)
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [postcondition], require)
                contract_patch_pre_post = veri.insert_empty_line(contract_patch_pre_post, precondition)
                contract_patch_pre_post = veri.replace_lines_with_string(contract_patch_pre_post, [precondition], require) 
                
                patch_lines = f"{precondition},{postcondition}"
                patch_pre_post = (contract_patch_pre_post, patch_lines, require)
                no_test, result_pre_post = veri.evaluate_contracts(contract, patch_pre_post)

                # Pre + VL + Post (PVP)
                if postcondition != contract_patch[1]: 
                        contract_patch_PVP = veri.insert_empty_line(contract_patch, postcondition)
                        contract_patch_PVP = veri.replace_lines_with_string(contract_patch_PVP, [postcondition], require)
                if precondition != contract_patch[1]: 
                        contract_patch_PVP = veri.insert_empty_line(contract_patch_PVP, precondition)
                        contract_patch_PVP= veri.replace_lines_with_string(contract_patch_PVP, [precondition], require) 

                patch_lines = f"{precondition},{contract_patch_VL[1]},{postcondition}"
                patch_PVP = (contract_patch_PVP, patch_lines, require)
                no_test, result_PVP = veri.evaluate_contracts(contract, patch_PVP)

                # Pre + VL (PV)
                if precondition != contract_patch[1]: 
                        contract_patch_PV = veri.insert_empty_line(contract_patch, precondition)
                        contract_patch_PV= veri.replace_lines_with_string(contract_patch_PV, [precondition], require) 

                patch_lines = f"{precondition},{contract_patch_VL[1]}"
                patch_PV = (contract_patch_PV, patch_lines, require)
                no_test, result_PV = veri.evaluate_contracts(contract, patch_PV)

                # VL + Post (VP)
                if postcondition != contract_patch[1]: 
                        contract_patch_VP = veri.insert_empty_line(contract_patch, postcondition)
                        contract_patch_VP = veri.replace_lines_with_string(contract_patch_VP, [postcondition], require)

                patch_lines = f"{contract_patch_VL[1]},{postcondition}"
                patch_VP = (contract_patch_VP, patch_lines, require)
                no_test, result_VP = veri.evaluate_contracts(contract, patch_VP)

            
                type_V = veri.get_directory_name(contract_name,"/home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated")
              

                row = {
                'Solidity_file_name': contract_name,
                'vulnerability_type': type_V,
                'vulnerable_line_number': contract_patch_VL[1],
                'vulnerability_function_entry_line': precondition - 1,
                'vulnerability_function_end_line': postcondition,
                'VL': result_VL,
                'pre_post_': result_pre_post,
                'pre': result_pre,
                'post': result_post,
                'pre_VL_post': result_PVP,
                'pre_VL': result_PV,
                'VL_post': result_VP
                }

                veri.append_row('/home/matteo/FLAMES/verification-results/sb-heists/reports/validation_results_20_8X.csv', headers, row)


        

CSV file '/home/matteo/FLAMES/verification-results/sb-heists/reports/validation_results_20_8X.csv' created successfully in Excel-friendly format.

=== Evaluating patches for contract: integer_overflow_1.sol ===

[Patch on line 13] Evaluation Results:
Inserted Require: require(value+sellerBalance>=sellerBalance);
Base directory: ../smartbugs-curated/0.4.x
Executing command:  npx hardhat compile
Nothing to compile


../smartbugs-curated/0.4.x
Source path: /home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated/0.8.x/contracts/dataset/arithmetic/integer_overflow_1.sol
Backing up file /home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated/0.8.x/contracts/dataset/arithmetic/integer_overflow_1.sol to ../smartbugs-curated/0.4.x/backups
Backup path: /home/matteo/FLAMES/verification-results/sb-heists/smartbugs-curated/0.8.x/contracts/dataset/arithmetic/integer_overflow_1.sol.bak
Running tests for contract: /home/matteo/FLAMES/verification-results/sb-heists/smartbugs-c

KeyboardInterrupt: 